In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os
import pandas as pd
import datetime as dt

In [2]:
expeditionsSheet_ID = '1rdE6YERXctYlLUICt1hPtJp3uLdZ7u9yeWcDQFJRHts'
expeditionsDataRange = 'Sheet1'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [3]:
creds = None
expCredsPath = 'expeditions.pkl'
if os.path.exists(expCredsPath):
    with open(expCredsPath, 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open(expCredsPath, 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [ ]:
result = sheet.values().get(spreadsheetId = expeditionsSheet_ID,
                           range=expeditionsDataRange).execute()
values = result.get('values', [])

In [ ]:
df = pd.DataFrame(values[1:], columns=values[0])
df['Link'][df['Link'] == ''] = None
df['Media'][df['Media'] == ''] = None
df['People'][df['People'] == ''] = None

In [ ]:
for index in df.index:
    df.loc[index, "Date"] = dt.datetime.strptime(df['Year'][index] + " "+ df['Month'][index], "%Y %B").date()

In [ ]:
df.sort_values("Date", inplace=True, ascending=False)

In [ ]:
html = ""
for index, row in df.iterrows():
    # heading
    html += "<!-- wp:heading -->\n"
    if row['Link']:
        html += "<h2>"
        html += "<a href=\"%s\" data-type=\"URL\" data-id=\"%s\">" % (row['Link'], row['Link'])
        html += "%s" % (row['Project'])
        html += "</a>"
        html += " - %s %s" % (row['Month'], row['Year'])
        
        html += "</h2>"
        html += "\n"
    else:
        html += "<h2>%s - %s %s</h2>\n" % (row['Project'], row['Month'], row['Year'])
    html += "<!-- /wp:heading -->\n"
    html += "\n"
    
    # Location
    html += "<!-- wp:paragraph -->\n"
    html += "<p><strong>Location:</strong> %s</p>\n" % (row["Location"])
    html += "<!-- /wp:paragraph -->\n"
    html += "\n"
    
    if row['People']:
        # People
    #     html += "<!-- wp:paragraph -->\n"
    #     html += "<p>People:</p>\n"
    #     html += "<!-- /wp:paragraph -->\n"
    #     html += "\n"

        html += "<!-- wp:list -->\n"
        html += "<ul>"
        for person in row['People'].splitlines():
            html += "<li>%s</li>" % (person)
        html += "</ul\n"
        html += "<!-- /wp:list -->\n"
        html += "\n"
print(html)
with open('expeditions.txt', 'w') as file:
    file.write(html)